In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import os,glob
from tqdm import tqdm
tqdm.pandas()
import shutil
import torch,torchvision
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import cv2
import warnings

warnings.filterwarnings('ignore')

from torch.cuda.amp import GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms
from torch.cuda.amp import autocast

In [25]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
!ls /content/drive/MyDrive/fer2013/models

In [28]:
class Block(nn.Module):
  def __init__(self, in_channels):
    super().__init__()
    self.in_channels = in_channels
    self.out_channels = (64 if self.in_channels == 1 else 2 * self.in_channels)

    self.block =  nn.Sequential(
        nn.Conv2d(self.in_channels, self.out_channels, kernel_size = 3, padding = 1),
        nn.BatchNorm2d(self.out_channels),
        nn.ReLU(inplace = True),
        nn.Conv2d(self.out_channels, self.out_channels, kernel_size = 3, padding = 1),
        nn.BatchNorm2d(self.out_channels),
        nn.ReLU(inplace = True),
        nn.MaxPool2d(kernel_size = 2, stride = 2),

    )

  def forward(self, x):
    out = self.block(x)
    return out

In [29]:
Block(64)

Block(
  (block): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)

In [30]:
class Vgg(nn.Module):
  def __init__(self, block, drop = 0.2, num_classes = 7):
    super().__init__()

    self.layer1 = self._make_layer(block, 1)
    self.layer2 = self._make_layer(block, 64)
    self.layer3 = self._make_layer(block, 128)
    self.layer4 = self._make_layer(block, 256)

    self.drop = nn.Dropout(p = drop)
    self.fc1 = nn.Linear(512 * 2 * 2, 4096)
    self.fc2 = nn.Linear(4096, 4096)
    self.fc3 = nn.Linear(4096, num_classes)

  def _make_layer(self, block, in_channels):
    return block(in_channels)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)

    out = torch.flatten(out, 1)
    out = F.relu(self.drop(self.fc1(out)))
    out = F.relu(self.drop(self.fc2(out)))
    out = self.fc3(out)

    return out


In [31]:
Vgg(Block)

Vgg(
  (layer1): Block(
    (block): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (layer2): Block(
    (block): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxP

In [32]:
from torch.utils.data import Dataset

In [33]:
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None, augment=False):
        self.images = images
        self.labels = labels
        self.transform = transform

        self.augment = augment

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img = np.array(self.images[idx])

        img = Image.fromarray(img)

        if self.transform:
            img = self.transform(img)

        label = torch.tensor(self.labels[idx]).type(torch.long)
        sample = (img, label)

        return sample

In [34]:
def load_data(path='/content/drive/MyDrive/fer2013/fer2013.csv'):
    fer2013 = pd.read_csv(path)
    emotion_mapping = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

    return fer2013, emotion_mapping


def prepare_data(data):
    """ Prepare data for modeling
        input: data frame with labels und pixel data
        output: image and label array """

    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['emotion'])))

    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, 'pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image

    return image_array, image_label


def get_dataloaders(path='/content/drive/MyDrive/fer2013/fer2013.csv', bs=64, augment=True):
    """ Prepare train, val, & test dataloaders
        Augment training data using:
            - cropping
            - shifting (vertical/horizental)
            - horizental flipping
            - rotation
        input: path to fer2013 csv file
        output: (Dataloader, Dataloader, Dataloader) """

    fer2013, emotion_mapping = load_data(path)

    xtrain, ytrain = prepare_data(fer2013[fer2013['Usage'] == 'Training'])
    xval, yval = prepare_data(fer2013[fer2013['Usage'] == 'PrivateTest'])
    xtest, ytest = prepare_data(fer2013[fer2013['Usage'] == 'PublicTest'])

    mu, st = 0, 255

    test_transform = transforms.Compose([
        # transforms.Scale(52),
        transforms.TenCrop(40),
        transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
        transforms.Lambda(lambda tensors: torch.stack([transforms.Normalize(mean=(mu,), std=(st,))(t) for t in tensors])),
    ])

    if augment:
        train_transform = transforms.Compose([
            transforms.RandomResizedCrop(48, scale=(0.8, 1.2)),
            transforms.RandomApply([transforms.RandomAffine(0, translate=(0.2, 0.2))], p=0.5),
            transforms.RandomHorizontalFlip(),
            transforms.RandomApply([transforms.RandomRotation(10)], p=0.5),

            transforms.TenCrop(40),
            transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
            transforms.Lambda(lambda tensors: torch.stack([transforms.Normalize(mean=(mu,), std=(st,))(t) for t in tensors])),
            transforms.Lambda(lambda tensors: torch.stack([transforms.RandomErasing(p=0.5)(t) for t in tensors])),
        ])
    else:
        train_transform = test_transform

    # X = np.vstack((xtrain, xval))
    # Y = np.hstack((ytrain, yval))

    train = CustomDataset(xtrain, ytrain, train_transform)
    val = CustomDataset(xval, yval, test_transform)
    test = CustomDataset(xtest, ytest, test_transform)

    trainloader = DataLoader(train, batch_size=bs, shuffle=True, num_workers=2)
    valloader = DataLoader(val, batch_size=64, shuffle=True, num_workers=2)
    testloader = DataLoader(test, batch_size=64, shuffle=True, num_workers=2)

    return trainloader, valloader, testloader

In [35]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def train(net, dataloader, criterion, optimizer, scaler, Ncrop=True):
    net = net.train()
    loss_tr, correct_count, n_samples = 0.0, 0.0, 0.0
    iters = len(dataloader)  # number of batches, not images

    for i, data in tqdm(enumerate(dataloader)):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # print(f'batch number: {i+1}')
        # print('before autocast')
        with autocast():
            if Ncrop:
                # fuse crops and batchsize
                # print('Ncrop started!!')
                bs, ncrops, c, h, w = inputs.shape
                inputs = inputs.view(-1, c, h, w)

            # print('Ncrop over!!')
            # repeat labels ncrops times
            labels = torch.repeat_interleave(labels, repeats=ncrops, dim=0)

            # print('repeat labels done!!')
            # forward + backward + optimize
            outputs = net(inputs)
            # print('model output taken!!')
            loss = criterion(outputs, labels)
            # print('loss calaculated!!')
            scaler.scale(loss).backward()
            # print('scaler scaled!!')

            scaler.step(optimizer)
            # print('scaler step!!')
            scaler.update()
            # print('scaler update done!!')
            optimizer.zero_grad()
            # print('optimizer weights cleared!!')
            # scheduler.step(epoch + i / iters)

            # calculate performance metrics
            loss_tr += loss.item()

            _, preds = torch.max(outputs.data, 1)
            correct_count += (preds == labels).sum().item()
            n_samples += labels.size(0)

    acc = 100 * correct_count / n_samples
    loss = loss_tr / n_samples

    return acc, loss


def evaluate(net, dataloader, criterion, Ncrop=True):
    net = net.eval()
    loss_tr, correct_count, n_samples = 0.0, 0.0, 0.0
    for data in tqdm(dataloader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        if Ncrop:
            # fuse crops and batchsize
            bs, ncrops, c, h, w = inputs.shape
            inputs = inputs.view(-1, c, h, w)
            # forward
            outputs = net(inputs)
            # combine results across the crops
            outputs = outputs.view(bs, ncrops, -1)
            outputs = torch.sum(outputs, dim=1) / ncrops
        else:
            outputs = net(inputs)

        loss = criterion(outputs, labels)

        # calculate performance metrics
        loss_tr += loss.item()

        _, preds = torch.max(outputs.data, 1)
        correct_count += (preds == labels).sum().item()
        n_samples += labels.size(0)

    acc = 100 * correct_count / n_samples
    loss = loss_tr / n_samples

    return acc, loss

In [36]:
import os

hps = {
    'network': '',  # which network do you want to train
    'name': '',  # whatever you want to name your run
    'n_epochs': 300,
    'model_save_dir': '/content/drive/MyDrive/fer2013/models',  # where will checkpoints be stored (path created automatically using hps[name])
    'restore_epoch': None,  # continue training from a specific saved point
    'start_epoch': 0,
    'lr': 0.01,  # starting learning rate
    'save_freq': 20,  # how often to create checkpoints
    'drop': 0.1,
    'bs': 64,
}

# possible_nets = set(filename.split(".")[0] for filename in os.listdir('models'))


def setup_hparams(args):
    for arg in args:
        key, value = arg.split('=')
        if key not in hps:
            raise ValueError(key + ' is not a valid hyper parameter')
        else:
            hps[key] = value

    # Invalid network check
    if hps['network'] not in possible_nets:
        raise ValueError("Invalid network.\nPossible ones include:\n - " + '\n - '.join(possible_nets))

    # invalid parameter check
    try:
        hps['n_epochs'] = int(hps['n_epochs'])
        hps['start_epoch'] = int(hps['start_epoch'])
        hps['save_freq'] = int(hps['save_freq'])
        hps['lr'] = float(hps['lr'])
        hps['drop'] = float(hps['drop'])
        hps['bs'] = int(hps['bs'])

        if hps['restore_epoch']:
            hps['restore_epoch'] = int(hps['restore_epoch'])
            hps['start_epoch'] = int(hps['restore_epoch'])

        # make sure we can checkpoint regularly or at least once (at the end)
        if hps['n_epochs'] < 20:
            hps['save_freq'] = min(5, hps['n_epochs'])

    except Exception as e:
        raise ValueError("Invalid input parameters")

    # create checkpoint directory
    # hps['model_save_dir'] = os.path.join(os.getcwd(), 'checkpoints', hps['name'])

    if not os.path.exists(hps['model_save_dir']):
        os.makedirs(hps['model_save_dir'])

    return hps

In [37]:
type(hps['bs'])

int

In [48]:
net = Vgg(Block)

In [49]:
net.to(device)
scaler = GradScaler()
optimizer = torch.optim.SGD(net.parameters(), lr=hps['lr'], momentum=0.9, nesterov=True, weight_decay=0.0001)

scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.75, patience=5, verbose=True)
criterion = nn.CrossEntropyLoss()


In [50]:
best_acc = 0.0

# net.to(device)
trainloader, valloader, testloader = get_dataloaders(bs = hps['bs'])
print("Training", "on", device)
for epoch in range(hps['start_epoch'], hps['n_epochs']):

  acc_tr, loss_tr = train(net, trainloader, criterion, optimizer, scaler)
  run['metrics/train_loss'].log(loss_tr)
  run['metrics/train_acc'].log(acc_tr)
        # logger.loss_train.append(loss_tr)
        # logger.acc_train.append(acc_tr)

  acc_v, loss_v = evaluate(net, valloader, criterion)
  run['metrics/val_loss'].log(loss_v)
  run['metrics/val_acc'].log(acc_v)
        # logger.loss_val.append(loss_v)
        # logger.acc_val.append(acc_v)

        # Update learning rate
  scheduler.step(acc_v)
  run['learning_rate'].log(optimizer.param_groups[0]['lr'])

  if acc_v > best_acc:
    best_acc = acc_v
    path = os.path.join(hps['model_save_dir'], 'epoch_' + str(epoch))
    torch.save(net.state_dict(), path)

    # save(net, logger, hps, epoch + 1)
            # logger.save_plt(hps)

  if (epoch + 1) % hps['save_freq'] == 0:
    path = os.path.join(hps['model_save_dir'], 'epoch_' + str(epoch))
    torch.save(net.state_dict(), path)
    # save(net, logger, hps, epoch + 1)
            # logger.save_plt(hps)

  checkpoint = {
      'epoch' : epoch,
      'model' : net.state_dict(),
      'optimizer' : optimizer.state_dict(),
      'scheduler' : scheduler,
      'scaler' : scaler.state_dict()
  }

  torch.save(checkpoint, '/content/drive/MyDrive/fer2013/checkpoints/model1.pth')

  print('Epoch %2d' % (epoch + 1),
              'Train Accuracy: %2.4f %%' % acc_tr,
              'Val Accuracy: %2.4f %%' % acc_v,
              sep='\t\t')

    # Calculate performance on test set
acc_test, loss_test = evaluate(net, testloader, criterion)
print('Test Accuracy: %2.4f %%' % acc_test,
          'Test Loss: %2.6f' % loss_test,
          sep='\t\t')

Training on cuda:0


449it [01:26,  5.19it/s]
100%|██████████| 57/57 [00:07<00:00,  8.10it/s]


Epoch  1		Train Accuracy: 34.2056 %		Val Accuracy: 50.0975 %


449it [01:25,  5.22it/s]
100%|██████████| 57/57 [00:07<00:00,  8.07it/s]


Epoch  2		Train Accuracy: 48.3270 %		Val Accuracy: 53.6082 %


...

Epoch 157		Train Accuracy: 93.6403 %		Val Accuracy: 73.4188 %


449it [01:23,  5.35it/s]
100%|██████████| 57/57 [00:07<00:00,  8.14it/s]


Epoch 158		Train Accuracy: 93.6173 %		Val Accuracy: 73.6695 %


360it [01:07,  5.30it/s]


KeyboardInterrupt: ignored

In [ ]:
!pip install neptune-client

In [40]:
import neptune.new as neptune

run = neptune.init(project = 'bavesh.balaji/Experiment-Tracking-Pytorch-fer2013',
                   api_token = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI5MTFkOTlhMi1iMTllLTRiZjgtYTI3Mi00NmNhMGY4NjIwMDQifQ==')

https://app.neptune.ai/bavesh.balaji/Experiment-Tracking-PyTorch-fer2013/e/EX-2
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [41]:
run['parameters'] = hps

In [51]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!


Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.


All 1 operations synced, thanks for waiting!
